In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
# Set the number of classes in your dataset
num_classes = 4

# Load the VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add your own classification layers on top of the base model
# Remove the bottom layers of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(64, activation='relu')(x)
x = Flatten()(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Set up the data generators for training and validation images
train_datagen = ImageDataGenerator(rescale=1.0 / 255,  # Rescale pixel values to [0, 1]
    # rotation_range=10, # rotation
    # width_shift_range=0.2, # horizontal shift
    # height_shift_range=0.2, # vertical shift
    # shear_range=0.2,  # Randomly apply shearing transformations
    # zoom_range=0.2,  # Randomly apply zooming transformations
    # horizontal_flip=True)  # Randomly flip images horizontally)
)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_dir = '/content/drive/MyDrive/Forest project/aug images/train'
validation_dir = '/content/drive/MyDrive/model data/validation'
batch_size = 16
num_epochs = 10

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

To get number of images present in train and validation folder

In [ ]:
import os

# folder path
tiger_p = '/content/drive/MyDrive/model data/validation/Tiger'
bear_p = '/content/drive/MyDrive/model data/validation/Bear'
leopard_p = '/content/drive/MyDrive/model data/validation/Leopard'
nothing_p = '/content/drive/MyDrive/model data/validation/Nothing'
# list of files in the folder
file_list_tig = os.listdir(tiger_p)
file_list_leo = os.listdir(leopard_p)
file_list_bear = os.listdir(bear_p)
file_list_not = os.listdir(nothing_p)


#  number of files
num_files_tig = len(file_list_tig)
num_files_leo = len(file_list_leo)
num_files_bear = len(file_list_bear)
num_files_not = len(file_list_not)


# Printing the number of files
print(f"Number of images of Tiger in the folder: {num_files_tig}")
print(f"Number of images of Bear in the folder: {num_files_bear}")
print(f"Number of images of Leopard in the folder: {num_files_leo}")
print(f"Number of images of Nothing in the folder: {num_files_not}")

Number of images of Tiger in the folder: 20
Number of images of Bear in the folder: 10
Number of images of Leopard in the folder: 19
Number of images of Nothing in the folder: 150


In [ ]:
 tiger_p = '/content/drive/MyDrive/Forest project/aug images/train/tiger'
bear_p = '/content/drive/MyDrive/Forest project/aug images/train/bear'
leopard_p = '/content/drive/MyDrive/Forest project/aug images/train/leopard'
nothing_p = '/content/drive/MyDrive/Forest project/aug images/train/nothing'
file_list_tig = os.listdir(tiger_p)
file_list_leo = os.listdir(leopard_p)
file_list_bear = os.listdir(bear_p)
file_list_not = os.listdir(nothing_p)
num_files_tig = len(file_list_tig)
num_files_leo = len(file_list_leo)
num_files_bear = len(file_list_bear)
num_files_not = len(file_list_not)
print(f"Number of images of Tiger in the folder: {num_files_tig}")
print(f"Number of images of Bear in the folder: {num_files_bear}")
print(f"Number of images of Leopard in the folder: {num_files_leo}")
print(f"Number of images of Nothing in the folder: {num_files_not}")

Number of images of Tiger in the folder: 201
Number of images of Bear in the folder: 92
Number of images of Leopard in the folder: 225
Number of images of Nothing in the folder: 2593


In [ ]:

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

Found 3111 images belonging to 4 classes.
Found 199 images belonging to 4 classes.


Training Model

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size)

Epoch 1/10
194/194 [==============================] - 2139s 11s/step - loss: 0.5896 - accuracy: 0.8278 - val_loss: 1.2218 - val_accuracy: 0.6562
Epoch 2/10
194/194 [==============================] - 63s 325ms/step - loss: 0.4547 - accuracy: 0.8346 - val_loss: 1.2688 - val_accuracy: 0.7552
Epoch 3/10
194/194 [==============================] - 67s 344ms/step - loss: 0.4410 - accuracy: 0.8330 - val_loss: 1.2182 - val_accuracy: 0.6875
Epoch 4/10
194/194 [==============================] - 65s 334ms/step - loss: 0.4152 - accuracy: 0.8420 - val_loss: 1.0998 - val_accuracy: 0.7240
Epoch 5/10
194/194 [==============================] - 64s 329ms/step - loss: 0.3896 - accuracy: 0.8478 - val_loss: 1.2670 - val_accuracy: 0.8177
Epoch 6/10
194/194 [==============================] - 64s 331ms/step - loss: 0.3693 - accuracy: 0.8627 - val_loss: 1.7210 - val_accuracy: 0.7135
Epoch 7/10
194/194 [==============================] - 62s 321ms/step - loss: 0.3619 - accuracy: 0.8659 - val_loss: 2.1979 - val_ac

In [ ]:

# Save the model
model.save('/content/drive/MyDrive/model save/model_four_classes_aug_image_9_6_model_new.h5')


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import io
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
data_dir = '/content/drive/MyDrive/Forest project/aug images/train'

class_labels = sorted(os.listdir(data_dir))
print(class_labels)

model prediction on test image from folder to be saved as dataframe

In [ ]:
image_folder = '/content/drive/MyDrive/test images/img_url'


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img

def predict(image_path):
    image = Image.open(image_path)
    image = preprocess_image(image_path)
    prediction = model.predict(image)
    return prediction

def predict_image_class(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction[0])
    predicted_class_index = np.argmax(prediction)
    return predicted_class_index

In [ ]:
# List to store the image names and predicted values
image_names = []
pred_index = []
all_predictions = []

# Iterate over the images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        # Load and preprocess the image
        image_path = os.path.join(image_folder, filename)
        prediction = predict(image_path)
        predicted_class_index = predict_image_class(image_path)
        pred_index.append(predicted_class_index)
        image_names.append(filename)
        all_predictions.append(prediction)

# Create a dataframe with image names and predicted values
data = {'Image Name': image_names,'predicted_class_index': pred_index ,'Predictions': all_predictions}
df = pd.DataFrame(data)

# Save the dataframe to a CSV file
output_file = 'predictions_test.csv'
df.to_csv(output_file, index=False)
print("Predictions saved to", output_file)

In [ ]:
import io
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

model prediction on test image 

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/content/drive/MyDrive/model.h5')

# Dictionary mapping class indices to class labels
# class_labels = {0: 'Leopard', 1: 'Nothing', 2: 'Tiger'}

# Function to preprocess the image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to make predictions on the image
def predict_image(image_path):
    preprocessed_img = preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(prediction)
    # predicted_label = class_labels[predicted_class_index]
    print(f"Predicted class: {predicted_label}")

# Path to the image you want to predict
image_path = '/content/images (12).jpeg'

# Make predictions on the image
predict_image(image_path)

Code to reolad the model after saving

In [ ]:
#model loading
model = load_model('/content/drive/MyDrive/model.h5')